<a href="https://colab.research.google.com/github/pmxfa/sp-shapely/blob/main/sp_timegan_weather.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install synthcity

# Training

In [2]:
from google.colab import drive
drive.mount('/content/drive')

import sys
import warnings
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

import synthcity.logger as log
from synthcity.plugins import Plugins
from synthcity.plugins.core.dataloader import TimeSeriesDataLoader
from synthcity.utils.serialization import save_to_file, load_from_file

log.add(sink=sys.stderr, level="INFO")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[KeOps] Compiling cuda jit compiler engine ... OK
[pyKeOps] Compiling nvrtc binder for python ... OK


## preprocessing

In [3]:
# Define file path
file_path = "/content/drive/Shareddrives/sp_env/datasets/Weather/weather.csv"

df = pd.read_csv(file_path)
print(df.head())
print(df.info())
print('missing values: \n', df.isnull().sum())

                  date  p (mbar)  T (degC)  Tpot (K)  Tdew (degC)  rh (%)  \
0  2020-01-01 00:10:00   1008.89      0.71    273.18        -1.33    86.1   
1  2020-01-01 00:20:00   1008.76      0.75    273.22        -1.44    85.2   
2  2020-01-01 00:30:00   1008.66      0.73    273.21        -1.48    85.1   
3  2020-01-01 00:40:00   1008.64      0.37    272.86        -1.64    86.3   
4  2020-01-01 00:50:00   1008.61      0.33    272.82        -1.50    87.4   

   VPmax (mbar)  VPact (mbar)  VPdef (mbar)  sh (g/kg)  ...  wv (m/s)  \
0          6.43          5.54          0.89       3.42  ...      1.02   
1          6.45          5.49          0.95       3.39  ...      0.43   
2          6.44          5.48          0.96       3.39  ...      0.61   
3          6.27          5.41          0.86       3.35  ...      1.11   
4          6.26          5.47          0.79       3.38  ...      0.49   

   max. wv (m/s)  wd (deg)  rain (mm)  raining (s)  SWDR (W/m�)  \
0           1.60     224.3     

In [4]:
# Set seed for reproducibility
np.random.seed(42)

# Convert 'date' to datetime and set as index (preserves chronological order)
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)
df.sort_index(inplace=True)

# Exclude 'date' from feature selection (it's now the index anyway)
columns = df.columns  # 'date' is no longer in df.columns
selected_features = np.random.choice(columns, size=10, replace=False)

print(f"Selected features: {selected_features}")

# Keep only the selected features
df = df[selected_features]

Selected features: ['p (mbar)' 'PAR (�mol/m�/s)' 'raining (s)' 'T (degC)' 'sh (g/kg)'
 'VPmax (mbar)' 'wv (m/s)' 'Tdew (degC)' 'max. PAR (�mol/m�/s)'
 'SWDR (W/m�)']


In [5]:
# Keep the latest 5000 rows
df_latest = df.tail(5000)

# Train-test split: 70% for training (for TimeGAN), 30% for testing (TSTR)
train_size = int(0.7 * len(df_latest))
df_train = df_latest.iloc[:train_size]
df_test = df_latest.iloc[train_size:]  # you might use this later for TSTR

# Normalize the data using MinMaxScaler
scaler = MinMaxScaler()
scaled_train = scaler.fit_transform(df_train)
df_scaled_train = pd.DataFrame(scaled_train, columns=df_train.columns, index=df_train.index)
scaled_test = scaler.transform(df_test)
df_scaled_test = pd.DataFrame(scaled_test, columns=df_test.columns, index=df_test.index)

# Sequence length for time-series data (dataset = per 10 mins, 36 = 6 hrs)
sequence_length = 36
temporal_data = []
observation_times = []

# Generate sequences from df_scaled_train only
for start in range(len(df_scaled_train) - sequence_length + 1):
    sequence = df_scaled_train.iloc[start:start + sequence_length].reset_index(drop=True)
    temporal_data.append(sequence)
    observation_times.append(list(range(sequence_length)))  # relative time within the window

# Dummy outcome for TimeGAN (can be used in DataLoader)
dummy_outcome = pd.DataFrame(np.zeros(len(temporal_data)), columns=["outcome"])

# --- Create DataLoader for TimeGAN ---
loader = TimeSeriesDataLoader(
    temporal_data=temporal_data,
    observation_times=observation_times,
    static_data=None,
    outcome=dummy_outcome,
)

# Print the loader info
print(f"TimeSeriesDataLoader created with {len(temporal_data)} sequences")

TimeSeriesDataLoader created with 3465 sequences


In [6]:
print(len(df_scaled_train))  # Check the length of the dataframe
print(loader.dataframe())

3500
        seq_id  seq_time_id  seq_temporal_PAR (�mol/m�/s)  \
0            0            0                      0.000000   
1            0            1                      0.000000   
2            0            2                      0.000000   
3            0            3                      0.000000   
4            0            4                      0.000000   
...        ...          ...                           ...   
124735    3464           31                      0.379520   
124736    3464           32                      0.396154   
124737    3464           33                      0.397787   
124738    3464           34                      0.451815   
124739    3464           35                      0.408922   

        seq_temporal_SWDR (W/m�)  seq_temporal_T (degC)  \
0                       0.000000               0.114042   
1                       0.000000               0.127230   
2                       0.000000               0.148177   
3                       0.

In [23]:
hparams = {
          "mode": "LSTM", # default mode = RNN;
          "dataloader_sampling_strategy": "none" # default = imbalanced_time_censoring
}`
# Load TimeGAN with custom parameters
syn_model = Plugins().get("timegan", **hparams)

[2025-05-07T10:35:02.239810+0000][3576][CRITICAL] module disabled: /usr/local/lib/python3.11/dist-packages/synthcity/plugins/generic/plugin_goggle.py


In [24]:
# --- Print all parameters of initialized model ---
for attr in dir(syn_model):
    if not attr.startswith("_") and not callable(getattr(syn_model, attr)):
        print(f"{attr}: {getattr(syn_model, attr)}")

batch_size: 64
class_name: TimeGANPlugin
clipping_value: 0
compress_dataset: False
dataloader_sampling_strategy: none
device: cuda
discriminator_batch_norm: False
discriminator_dropout: 0.1
discriminator_loss: None
discriminator_lr: 0.001
discriminator_n_iter: 1
discriminator_n_layers_hidden: 3
discriminator_n_units_hidden: 300
discriminator_nonlin: leaky_relu
discriminator_weight_decay: 0.001
embedding_penalty: 10
encoder: None
encoder_max_clusters: 20
expecting_conditional: False
fitted: False
gamma_penalty: 1
generator_batch_norm: False
generator_dropout: 0.01
generator_loss: None
generator_lr: 0.001
generator_n_layers_hidden: 2
generator_n_units_hidden: 150
generator_nonlin: leaky_relu
generator_nonlin_out_continuous: tanh
generator_nonlin_out_discrete: softmax
generator_residual: True
generator_weight_decay: 0.001
mode: LSTM
module_name: synthcity.plugins.time_series.plugin_timegan
module_relative_path: ../time_series/plugin_timegan.py
moments_penalty: 100
n_iter: 1000
n_iter_prin

## fitting the model

In [25]:
print(loader.shape)

# Train the model
syn_model.fit(loader)

(124740, 13)


100%|██████████| 1000/1000 [2:53:03<00:00, 10.38s/it]


In [26]:
#@title optional: generate data from saved model
from synthcity.utils.serialization import save_to_file, load_from_file

# Save model to drive
save_to_file('/content/drive/Shareddrives/sp_env/saved_models/GAN_Weather.pkl', syn_model)

In [27]:
# --- Generate Synthetic Data ---
n_samples = len(temporal_data)
print("n_samples:", n_samples)
syn_data = syn_model.generate(count=n_samples)
print(syn_data.shape)

n_samples: 3465
(75161, 13)


In [28]:
# --- Save with automated format ---
import datetime
import os
# Get the current date and time
now = datetime.datetime.now()
timestamp = now.strftime("%m%d%y-%H%M%S")  # MMDDYY-HHMMSS format

# Define the base directory
base_dir = "/content/drive/Shareddrives/sp_env/synthetic_datasets/TimeGAN/weather"  #CHANGE THIS
if not os.path.exists(base_dir):
    os.makedirs(base_dir)

# Construct the filename
model_name = type(syn_model).__name__.lower() # Get model name dynamically
filename = f"{timestamp}-{model_name}-n_3500.csv"
filepath = os.path.join(base_dir, filename)

# Save the data
df_syn = syn_data.dataframe()
df_syn.to_csv(filepath, index=False)

print(f"Synthetic data saved to: {filepath}")

Synthetic data saved to: /content/drive/Shareddrives/sp_env/synthetic_datasets/TimeGAN/weather/050725-134006-timeganplugin-n_3500.csv


# Evaluation

## Prerequisites

In [29]:
loader

,seq_id,seq_time_id,seq_temporal_PAR (�mol/m�/s),seq_temporal_SWDR (W/m�),seq_temporal_T (degC),seq_temporal_Tdew (degC),seq_temporal_VPmax (mbar),seq_temporal_max. PAR (�mol/m�/s),seq_temporal_p (mbar),seq_temporal_raining (s),seq_temporal_sh (g/kg),seq_temporal_wv (m/s),seq_out_outcome
0,0,0,0.000000,0.000000,0.114042,0.391513,0.077118,0.000000,0.837838,0.0,0.292105,0.132780,0.0
1,0,1,0.000000,0.000000,0.127230,0.397918,0.087231,0.000000,0.837838,0.0,0.297368,0.055325,0.0
2,0,2,0.000000,0.000000,0.148177,0.403523,0.102402,0.000000,0.840744,0.0,0.302632,0.051176,0.0
3,0,3,0.000000,0.000000,0.150504,0.403523,0.103666,0.000000,0.841325,0.0,0.302632,0.110650,0.0
4,0,4,0.000000,0.000000,0.149728,0.409127,0.103666,0.000000,0.839872,0.0,0.307895,0.087137,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
124735,3464,31,0.379520,0.341668,0.576416,0.741393,0.479140,0.332190,0.856437,0.0,0.655263,0.412172,0.0
124736,3464,32,0.396154,0.359281,0.577192,0.742994,0.479140,0.339379,0.855856,0.0,0.657895,0.420470,0.0
124737,3464,33,0.397787,0.362148,0.577967,0.744596,0.480405,0.349280,0.854693,0.0,0.657895,0.388658,0.0
124738,3464,34,0.451815,0.419480,0.589604,0.751001,0.493047,0.385717,0.854403,0.0,0.665789,0.271093,0.0


In [30]:
# Define selected columns explicitly
selected_columns = ['seq_temporal_PAR (�mol/m�/s)', 'seq_temporal_SWDR (W/m�)', 'seq_temporal_T (degC)',	'seq_temporal_Tdew (degC)',	'seq_temporal_VPmax (mbar)',	'seq_temporal_max. PAR (�mol/m�/s)',	'seq_temporal_p (mbar)',	'seq_temporal_raining (s)',	'seq_temporal_sh (g/kg)',	'seq_temporal_wv (m/s)']
# Ensure real_data and synthetic_data only contain the selected columns
real_data = loader.dataframe()[selected_columns].to_numpy()
synthetic_data = syn_data.dataframe()[selected_columns].to_numpy()

In [31]:
print(real_data, "\n ------------------------------------------------------- \n", synthetic_data)
print(type(real_data),type(synthetic_data))
print(real_data.shape,synthetic_data.shape)

[[0.         0.         0.11404189 ... 0.         0.29210526 0.13278008]
 [0.         0.         0.12723041 ... 0.         0.29736842 0.05532503]
 [0.         0.         0.14817688 ... 0.         0.30263158 0.05117566]
 ...
 [0.39778664 0.36214804 0.57796742 ... 0.         0.65789474 0.38865837]
 [0.45181468 0.41948019 0.58960434 ... 0.         0.66578947 0.27109267]
 [0.40892219 0.37985731 0.60512025 ... 0.         0.67631579 0.31258645]] 
 ------------------------------------------------------- 
 [[0.00000000e+00 0.00000000e+00 6.59008511e-01 ... 0.00000000e+00
  4.20039350e-01 1.35567956e-01]
 [3.76885321e-05 0.00000000e+00 2.42707856e-01 ... 0.00000000e+00
  5.04569243e-01 1.35470773e-01]
 [9.50433284e-06 0.00000000e+00 2.42746615e-01 ... 2.72382253e-06
  5.04461608e-01 1.35184309e-01]
 ...
 [7.69884213e-05 0.00000000e+00 2.92376809e-01 ... 1.47643289e-05
  6.42898767e-01 7.83156616e-02]
 [8.08675309e-05 0.00000000e+00 6.56675326e-01 ... 8.22913082e-06
  6.42783858e-01 2.51136509e-

## Generate distance metrics

### Helper Functions

In [32]:
from scipy.stats import wasserstein_distance, entropy
import numpy as np

def compute_wasserstein(real_data, synthetic_data, selected_columns):
    """
    Computes Wasserstein Distance between real and synthetic time-series data.

    """

    # Ensure both datasets have the same number of samples
    min_length = min(len(real_data), len(synthetic_data))
    real_trimmed = real_data[:min_length]  # Keep original order (no random sampling)
    synthetic_trimmed = synthetic_data[:min_length]  # Match size
    print(real_trimmed.shape,synthetic_trimmed.shape)

    wasserstein_results = {}

    # Compute Wasserstein Distance for each feature
    for i, col in enumerate(selected_columns):
        w_dist = wasserstein_distance(real_trimmed[:, i], synthetic_trimmed[:, i])
        wasserstein_results[col] = w_dist
        print(f"{w_dist}")

    return wasserstein_results

def compute_kl_divergence(real_data, synthetic_data, selected_columns, bins=50):
    """
    Computes KL Divergence between real and synthetic time-series data.

    """

    # Ensure both datasets have the same number of samples
    min_length = min(len(real_data), len(synthetic_data))
    real_trimmed = real_data[:min_length]  # Keep original order
    synthetic_trimmed = synthetic_data[:min_length]  # Match size

    kl_results = {}

    for i, col in enumerate(selected_columns):
        # Compute histogram-based probability distributions
        real_hist, _ = np.histogram(real_trimmed[:, i], bins=bins, density=True)
        synth_hist, _ = np.histogram(synthetic_trimmed[:, i], bins=bins, density=True)

        # Avoid zero probabilities (KL Divergence is undefined for zero values)
        real_hist += 1e-10
        synth_hist += 1e-10

        # Compute KL Divergence
        kl_div = entropy(real_hist, synth_hist)
        kl_results[col] = kl_div
        print(f"{kl_div}")

    return kl_results

### Generate Metrics

In [33]:
# Assuming df_scaled is the DataFrame containing your scaled ETD data

# Compute Wasserstein Distance
wasserstein_results = compute_wasserstein(real_data, synthetic_data, selected_columns)
print("Wasserstein Distance Results:")
print(wasserstein_results)

# Compute KL Divergence
kl_results = compute_kl_divergence(real_data, synthetic_data, selected_columns)
print("KL Divergence Results:")
print(kl_results)

(75161, 10) (75161, 10)
0.07775767367037037
0.07297001506072363
0.08057437347286017
0.14067274022480525
0.046833704824411786
0.06984135454614405
0.26704808604418634
0.03534949833933424
0.11284320252244102
0.034651670627721703
Wasserstein Distance Results:
{'seq_temporal_PAR (�mol/m�/s)': 0.07775767367037037, 'seq_temporal_SWDR (W/m�)': 0.07297001506072363, 'seq_temporal_T (degC)': 0.08057437347286017, 'seq_temporal_Tdew (degC)': 0.14067274022480525, 'seq_temporal_VPmax (mbar)': 0.046833704824411786, 'seq_temporal_max. PAR (�mol/m�/s)': 0.06984135454614405, 'seq_temporal_p (mbar)': 0.26704808604418634, 'seq_temporal_raining (s)': 0.03534949833933424, 'seq_temporal_sh (g/kg)': 0.11284320252244102, 'seq_temporal_wv (m/s)': 0.034651670627721703}
4.351929148919477
4.593972739912829
11.371749080496096
14.123927023848248
10.343157330912709
3.6271750571125065
10.7397138238509
0.6057989368185298
13.407676944585818
10.602265927820701
KL Divergence Results:
{'seq_temporal_PAR (�mol/m�/s)': 4.3519

# LSTM downstream

In [34]:
real_data = df_scaled_test
df_synth = pd.read_csv(filepath)

# 2. Drop the unwanted column
real_data = real_data.drop(columns=["seq_id", "seq_time_id", "seq_out_outcome"], errors="ignore")
df_synth = df_synth.drop(columns=["seq_id", "seq_time_id", "seq_out_outcome"], errors="ignore")

In [35]:
print(f"real_data: {real_data.shape}, synthetic_data: {df_synth.shape}")

real_data: (1500, 10), synthetic_data: (75161, 10)


In [36]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [37]:
# Convert to tensors (float32 for PyTorch)
data_real = torch.tensor(real_data.values, dtype=torch.float32)
data_synth = torch.tensor(df_synth.values, dtype=torch.float32)

# ──────── Sequence builder ───────────
def make_sequences(data, seq_len):
    X, y = [], []
    for i in range(len(data) - seq_len):
        X.append(data[i:i+seq_len])
        y.append(data[i+seq_len])
    return torch.stack(X), torch.stack(y)

SEQ_LEN = sequence_length

# Sequences for synthetic (train)
X_train, y_train = make_sequences(data_synth, SEQ_LEN)
train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=32, shuffle=True)

# Sequences for real (test)
X_test, y_test = make_sequences(data_real, SEQ_LEN)

In [38]:
# ─── Model Definition ──────────────────────────────────────
class ShallowLSTM(nn.Module):
    def __init__(self, input_size, hidden_size=64):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.linear = nn.Linear(hidden_size, input_size)

    def forward(self, x):
        _, (hn, _) = self.lstm(x)  # hn shape: (1, batch, hidden_size)
        out = self.linear(hn.squeeze(0))  # squeeze to (batch, hidden_size)
        return out


# ─── Model Init ─────────────────────────────────────────────
model = ShallowLSTM(input_size=X_train.shape[2], hidden_size=64)

# ─── Optimizer & Loss ───────────────────────────────────────
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.002)

# ─── Training ───────────────────────────────────────────────
EPOCHS = 50
for epoch in range(1, EPOCHS + 1):
    model.train()
    for xb, yb in train_loader:
        pred = model(xb)
        loss = loss_fn(pred, yb)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # if epoch % 10 == 0 or epoch == 1:
    print(f"Epoch {epoch}: Train MSE = {loss.item():.6f}")

Epoch 1: Train MSE = 0.011018
Epoch 2: Train MSE = 0.012898
Epoch 3: Train MSE = 0.011447
Epoch 4: Train MSE = 0.005740
Epoch 5: Train MSE = 0.010357
Epoch 6: Train MSE = 0.006060
Epoch 7: Train MSE = 0.006814
Epoch 8: Train MSE = 0.008539
Epoch 9: Train MSE = 0.007234
Epoch 10: Train MSE = 0.007810
Epoch 11: Train MSE = 0.007303
Epoch 12: Train MSE = 0.014812
Epoch 13: Train MSE = 0.008603
Epoch 14: Train MSE = 0.007567
Epoch 15: Train MSE = 0.014110
Epoch 16: Train MSE = 0.014633
Epoch 17: Train MSE = 0.009424
Epoch 18: Train MSE = 0.013965
Epoch 19: Train MSE = 0.007827
Epoch 20: Train MSE = 0.009334
Epoch 21: Train MSE = 0.007834
Epoch 22: Train MSE = 0.012117
Epoch 23: Train MSE = 0.007311
Epoch 24: Train MSE = 0.007792
Epoch 25: Train MSE = 0.008007
Epoch 26: Train MSE = 0.011993
Epoch 27: Train MSE = 0.013704
Epoch 28: Train MSE = 0.011806
Epoch 29: Train MSE = 0.010382
Epoch 30: Train MSE = 0.010657
Epoch 31: Train MSE = 0.007499
Epoch 32: Train MSE = 0.007853
Epoch 33: Train M

In [39]:
model.eval()
with torch.no_grad():
    preds = model(X_test)
    test_mse = loss_fn(preds, y_test).item()
    test_mae = mean_absolute_error(y_test.numpy(), preds.numpy())

    print(f"Test MSE: {test_mse:.6f}")
    print(f"Test MAE: {test_mae:.6f}")

Test MSE: 0.439717
Test MAE: 0.472249
